In [1]:
package.path = '../?.lua;'..package.path
require 'torch'
require 'nn'
require 'image'

require 'densecap.DenseCapModel'
utils = require 'densecap.utils'
box_utils = require 'densecap.box_utils'
vis_utils = require 'densecap.vis_utils'

In [2]:
-- options
checkpoint = '../data/models/densecap/densecap-pretrained-vgg16.t7'
image_size = 720
img_path = '../imgs/elephant.jpg'
box_width = 2
-- load model
dtype, use_cudnn = utils.setup_gpus(0, 1)
print(dtype, use_cudnn)
model = torch.load(checkpoint).model
model:convert(dtype, use_cudnn)
model:evaluate()

...g/Developments/torch/install/share/lua/5.1/cudnn/ffi.lua:1278: 'libcudnn (R4) not found in library path.
Please install CuDNN from https://developer.nvidia.com/cuDNN
Then make sure files named as libcudnn.so.4 or libcudnn.4.dylib are placed in your library load path (for example /usr/local/lib , or manually add a path to LD_LIBRARY_PATH)

stack traceback:
	[C]: in function 'error'
	...g/Developments/torch/install/share/lua/5.1/cudnn/ffi.lua:1278: in main chunk
	[C]: in function 'require'
	.../Developments/torch/install/share/lua/5.1/cudnn/init.lua:4: in main chunk
	[C]: in function 'require'
	../densecap/utils.lua:31: in function 'setup_gpus'
	[string "-- options..."]:7: in main chunk
	[C]: in function 'xpcall'
	...Developments/torch/install/share/lua/5.1/itorch/main.lua:209: in function <...Developments/torch/install/share/lua/5.1/itorch/main.lua:173>
	...Developments/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...elopments/torch/install/share/lua/5.1/lzmq/i

In [54]:
function view_box(box)
    x1 = torch.floor(box[1]-box[3]/2+1)
    y1 = torch.floor(box[2]-box[4]/2+1)
    x2 = x1 + box[3] - 1
    y2 = y1 + box[4] - 1
    itorch.image(img[{ {}, {y1, y2}, {x1, x2} }])
end
-- read image and box
img = image.load(img_path, 3)
box1 = torch.Tensor{400, 260, 400, 300}
box2 = torch.Tensor{380, 110, 120, 200}
-- box = torch.Tensor{400, 90, 100, 100}
view_box(box1)
view_box(box2)
boxes = torch.cat({box1, box2},1):view(2, -1)

In [55]:
-- rescale box and img
boxes = boxes:mul(image_size/math.max(img:size(2), img:size(3)))
img = image.scale(img, image_size):float()
H, W = img:size(2), img:size(3)
-- convert img
img_caffe = img:view(1, 3, H, W)
img_caffe = img_caffe:index(2, torch.LongTensor{3,2,1}):mul(255)
vgg_mean = torch.FloatTensor{103.939, 116.779, 123.68}
vgg_mean = vgg_mean:view(1, 3, 1, 1):expand(1, 3, H, W)
img_caffe:add(-1, vgg_mean)

In [56]:
out = model:forward_boxes(img_caffe:type(dtype), boxes:type(dtype))
obj_scores, seqs, roi_codes, hidden_codes, captions = unpack(out)
print('object score: ', obj_scores)
print(captions)
print(roi_codes:size(), roi_codes:type())
print(hidden_codes:size(), hidden_codes:type())
print(seqs:size(), seqs:type())

...evelopments/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.Sequential:
In 1 module of nn.Sequential:
...ts/torch/install/share/lua/5.1/nn/SpatialConvolution.lua:111: attempt to index field 'THNN' (a nil value)
stack traceback:
	...ts/torch/install/share/lua/5.1/nn/SpatialConvolution.lua:111: in function <...ts/torch/install/share/lua/5.1/nn/SpatialConvolution.lua:107>
	[C]: in function 'xpcall'
	...evelopments/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...velopments/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function <...velopments/torch/install/share/lua/5.1/nn/Sequential.lua:41>
	[C]: in function 'xpcall'
	...evelopments/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...velopments/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'forward'
	../densecap/DenseCapModel.lua:253: in function 'forward'
	../densecap/DenseCapModel.lua:321: in function 'forward_test'
	[string "-- fo

In [ ]:
-- -- forward model
-- boxes, scores, captions = model:forward_test(img_caffe:type(dtype))